In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from skimage.feature import hog
import time
import matplotlib.image as mpimg

#### Loading dataset

In [2]:
cars_dir = [car for car in glob.glob("../dataset/vehicles/*/*.png",recursive=True)]
non_car_dir = [non_car for non_car in glob.glob("../dataset/non-vehicles/*/*.png",recursive=True)]
print("Cars loaded: {} Non cars loaded: {}".format(len(cars_dir),len(non_car_dir)))

Cars loaded: 8792 Non cars loaded: 8968


## HOG

#### Feature Extraction 

In [3]:
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
#     print(features.shape)
    return features

def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
#     Seems like a useless step each channel for all images return same result
#     plt.hist(channel1_hist,bins=32,range=(0,256))
#     plt.hist(channel2_hist,bins=32,range=(0,256))
#     plt.hist(channel3_hist,bins=32,range=(0,256))
#     print(hist_features.shape)
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Return HOG features and visualization
# Hog takes only gray images
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualize=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualize=vis, feature_vector=feature_vec)
        return features
def convert_rgb_color(img, conv='YCrCb'):
    if conv == 'RGB':
        return np.copy(img)
    if conv == 'YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'HSV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    if conv == 'HLS':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    if conv == 'LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    if conv == 'YUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    
# Extract feature wrapper that extracts and combines all features
def extract_features(imgs,params):
    cspace= params['color_space']
    hog_channel=params['hog_channel']
    orient=params['orient']
    pix_per_cell= params['pix_per_cell']
    cell_per_block=params['cell_per_block']
    spatial_size=params['spatial_size']
    hist_bins=params['hist_bins']
    hist_range=params['hist_range']
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB' 
        feature_image = convert_rgb_color(image,cspace)
        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features) #Join all channels hog features into one feature vector        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range) # Seems useless (?)
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features,hog_features))) # Add all features into one feature vector
        
    # Return list of feature vectors
    return features

In [4]:
## Parameters
params={}
params['color_space'] ='YCrCb'
params['hog_channel'] ='ALL'
params['orient'] = 9
params['pix_per_cell'] = 8
params['cell_per_block'] = 2
params['spatial_size'] = (32,32)
params['hist_bins'] = 32 
params['hist_range'] = (0,256)

## SVM 

#### Extracting features

In [5]:
t1=time.time()

cars_features = extract_features(cars_dir,params)
notcars_features = extract_features(non_car_dir,params) 

t2 = time.time()
print(round(t2-t1, 2), 'second to extract features (HOG,spatial and color features).')

672.68 second to extract features (HOG,spatial and color features).


#### Data Standardization Step

In [6]:
#Making one feature vector that starts with cars and ends with non_cars
X = np.vstack((cars_features, notcars_features)).astype(np.float64)
#Computes the mean and standard deviation to be used for later scaling.
X_scaler = StandardScaler().fit(X)  #z = (x - u) / s

# Apply the scaler to X
scaled_X = X_scaler.transform(X) #Perform standardization by centering and scaling.

# scaled_X = StandardScaler().fit_transform(X) #  Does both steps at once
# Labeling feature vector <Cars = 1> <Not Cars = 0>
y = np.hstack((np.ones(len(cars)), np.zeros(len(notcars))))

NameError: name 'StandardScaler' is not defined